#KOD

##POMOCNICZE KLASY DO WCZYTYWANIA DANYCH

In [ ]:
import numpy as np


def _get_score(x_instance, W):
    return np.dot(W, x_instance)


def cross_entropy_loss_function(x_instance, W, y_i, debug=False):
    s = _get_score(x_instance, W)
    if debug:
        print('SCORE:', s)
        e_s = np.e ** s
        print('e^s: ', e_s)
        e_s_sum = np.sum(e_s)
        print('sum of the e^s:', e_s_sum)
        normalized_probability = np.e ** s[y_i] / e_s_sum
        print('normalized probability:', normalized_probability)
        result = -np.log(normalized_probability)
        print('loss:', result)
        return result
    return -np.log(np.e ** s[y_i] / np.sum(np.e ** s))


def loss_function_for_the_dataset(x, W, y, loss_fn=cross_entropy_loss_function):
    n = len(x)
    loss_sum = 0.0
    for i in range(n):
        loss_sum += loss_fn(x[i], W, y[i])
    return loss_sum/n


def loss_function_for_the_dataset_with_L2(x, W, y, loss_fn=cross_entropy_loss_function, regularization_strength=0.01):
    loss = loss_function_for_the_dataset(x, W, y, loss_fn)
    return loss + regularization_strength * np.sum(W**2)


# numerical gradient
def compute_gradient(training_x, training_y, W, h=0.001, regularization_strength=0.01):
    loss0 = loss_function_for_the_dataset_with_L2(training_x, W, training_y, regularization_strength=regularization_strength)
    grad = np.empty(W.shape)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W[i, j] += h
            loss = loss_function_for_the_dataset_with_L2(training_x, W, training_y, regularization_strength=regularization_strength)
            grad[i, j] = (loss - loss0)/h
            W[i, j] -= h
    return grad


class LinearClassifier:
    # weight matrix is by default np.ones
    def __init__(self, training_x, training_y, test_x, test_y, classes_length):
        self.training_x = training_x
        self.training_y = training_y
        self.test_x = test_x
        self.test_y = test_y
        self.W = np.ones((classes_length, training_x.shape[1]))

    def get_score(self, x_instance):
        return _get_score(x_instance, self.W)

    def random_search(self, iterations=1000, regularization_strength=0.01):
        self.W = np.ones(self.W.shape)
        best_loss = loss_function_for_the_dataset_with_L2(self.training_x, self.W, self.training_y, regularization_strength=regularization_strength)
        for num in range(0, iterations):
            weight_matrix = np.random.randn(*self.W.shape) * 0.001  # from Michigan Online
            loss = loss_function_for_the_dataset_with_L2(self.training_x, weight_matrix, self.training_y, regularization_strength=regularization_strength)
            if loss < best_loss:
                best_loss = loss
                self.W = weight_matrix
        return self.W, best_loss

    def check_accuracy(self):
        correct = 0
        options = [0, 0, 0]
        for i in range(0, self.test_x.shape[0]):
            result = self.get_score(self.test_x[i])
            selected_class = np.argmax(result)
            options[selected_class] += 1
            if selected_class == self.test_y[i]:
                correct = correct + 1

        return correct / self.test_x.shape[0], options

    def gradient_descent(self, learning_rate=0.01, h=0.001, iterations=1000, regularization_strength=0.01):
        self.W = np.ones(self.W.shape)  # init weight matrix
        for i in range(iterations):
            grad = compute_gradient(self.training_x, self.training_y, self.W, h, regularization_strength)
            self.W -= learning_rate * grad
        return self.W, loss_function_for_the_dataset_with_L2(self.training_x, self.W, self.training_y, regularization_strength=regularization_strength)

    def adam(self, learning_rate=0.001, h=0.001, beta1=0.9, beta2=0.999, iterations=1000, regularization_strength=0.01):
        self.W = np.ones(self.W.shape)  # init weight matrix
        moment1 = 0
        moment2 = 0
        for t in range(1, iterations + 1):
            dw = compute_gradient(self.training_x, self.training_y, self.W, h, regularization_strength)
            moment1 = beta1 * moment1 + (1 - beta1) * dw
            moment2 = beta2 * moment2 + (1 - beta2) * (dw ** 2)
            moment1_unbias = moment1 / (1 - beta1 ** t)
            moment2_unbias = moment2 / (1 - beta2 ** t)
            self.W -= learning_rate * moment1_unbias / (np.sqrt(moment2_unbias) + 1e-7)
        return self.W, loss_function_for_the_dataset_with_L2(self.training_x, self.W, self.training_y, regularization_strength=regularization_strength)


##FUNKCJE I KLASA ODPOWIEDZIALNA NA LINIOWY KLASYFIKATOR

In [ ]:
def _get_score(x_instance, W):
    return np.dot(W, x_instance)


def cross_entropy_loss_function(x_instance, W, y_i, debug=False):
    s = _get_score(x_instance, W)
    if debug:
        print('SCORE:', s)
        e_s = np.e ** s
        print('e^s: ', e_s)
        e_s_sum = np.sum(e_s)
        print('sum of the e^s:', e_s_sum)
        normalized_probability = np.e ** s[y_i] / e_s_sum
        print('normalized probability:', normalized_probability)
        result = -np.log(normalized_probability)
        print('loss:', result)
        return result
    return -np.log(np.e ** s[y_i] / np.sum(np.e ** s))


def loss_function_for_the_dataset(x, W, y, loss_fn=cross_entropy_loss_function):
    n = len(x)
    loss_sum = 0.0
    for i in range(n):
        loss_sum += loss_fn(x[i], W, y[i])
    return loss_sum/n


def loss_function_for_the_dataset_with_L2(x, W, y, loss_fn=cross_entropy_loss_function, regularization_strength=0.01):
    loss = loss_function_for_the_dataset(x, W, y, loss_fn)
    return loss + regularization_strength * np.sum(W**2)


# numerical gradient
def compute_gradient(training_x, training_y, W, h=0.001, regularization_strength=0.01):
    loss0 = loss_function_for_the_dataset_with_L2(training_x, W, training_y, regularization_strength=regularization_strength)
    grad = np.empty(W.shape)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W[i, j] += h
            loss = loss_function_for_the_dataset_with_L2(training_x, W, training_y, regularization_strength=regularization_strength)
            grad[i, j] = (loss - loss0)/h
            W[i, j] -= h
    return grad


class LinearClassifier:
    # weight matrix is by default np.ones
    def __init__(self, training_x, training_y, test_x, test_y, classes_length):
        self.training_x = training_x
        self.training_y = training_y
        self.test_x = test_x
        self.test_y = test_y
        self.W = np.ones((classes_length, training_x.shape[1]))

    def get_score(self, x_instance):
        return _get_score(x_instance, self.W)

    def random_search(self, iterations=1000, regularization_strength=0.01):
        self.W = np.ones(self.W.shape)
        best_loss = loss_function_for_the_dataset_with_L2(self.training_x, self.W, self.training_y, regularization_strength=regularization_strength)
        for num in range(0, iterations):
            weight_matrix = np.random.randn(*self.W.shape) * 0.001  # from Michigan Online
            loss = loss_function_for_the_dataset_with_L2(self.training_x, weight_matrix, self.training_y, regularization_strength=regularization_strength)
            if loss < best_loss:
                best_loss = loss
                self.W = weight_matrix
        return self.W, best_loss

    def check_accuracy(self):
        correct = 0
        options = [0, 0, 0]
        for i in range(0, self.test_x.shape[0]):
            result = self.get_score(self.test_x[i])
            selected_class = np.argmax(result)
            options[selected_class] += 1
            if selected_class == self.test_y[i]:
                correct = correct + 1

        return correct / self.test_x.shape[0], options

    def gradient_descent(self, learning_rate=0.01, h=0.001, iterations=1000, regularization_strength=0.01):
        self.W = np.ones(self.W.shape)  # init weight matrix
        for i in range(iterations):
            grad = compute_gradient(self.training_x, self.training_y, self.W, h, regularization_strength)
            self.W -= learning_rate * grad
        return self.W, loss_function_for_the_dataset_with_L2(self.training_x, self.W, self.training_y, regularization_strength=regularization_strength)

    def adam(self, learning_rate=0.001, h=0.001, beta1=0.9, beta2=0.999, iterations=1000, regularization_strength=0.01):
        self.W = np.ones(self.W.shape)  # init weight matrix
        moment1 = 0
        moment2 = 0
        for t in range(1, iterations + 1):
            dw = compute_gradient(self.training_x, self.training_y, self.W, h, regularization_strength)
            moment1 = beta1 * moment1 + (1 - beta1) * dw
            moment2 = beta2 * moment2 + (1 - beta2) * (dw ** 2)
            moment1_unbias = moment1 / (1 - beta1 ** t)
            moment2_unbias = moment2 / (1 - beta2 ** t)
            self.W -= learning_rate * moment1_unbias / (np.sqrt(moment2_unbias) + 1e-7)
        return self.W, loss_function_for_the_dataset_with_L2(self.training_x, self.W, self.training_y, regularization_strength=regularization_strength)


# TESTY

##TEST #0 - WCZYTANIE DANYCH

In [ ]:
iris = CSVReader('drive/MyDrive/ML/IRIS.csv')

print('RAW DATA')
print(iris.get_data())

print('RAW DATA AS NUMPY')
data = iris.get_numpy()
print(data)

manager = DataManager(data)
print("SEPARATED X AND Y")
x, y = manager.separate_x_y()
print('X')
print(x)
print('Y')
print(y)

print('CONVERTED X TO FLOATS')
x = manager.convert_x_to_floats()
print(x)

print('CONVERTED CLASSES NAMES TO IDS')
convention, y = manager.convert_classes_to_numbers()
print(y)
print(convention)
classes = [x[1] for x in convention]

print('BIAS TRICK')
x = manager.add_bias_trick()
print(x)

print('TRAINING AND TEST SETS')
training_x, training_y, test_x, test_y = manager.get_training_test_set_in_equal_portion()
print('TRAINING SET')
print(training_x, training_y)
print('TEST SET (20% OF THE DATASET)')
print(test_x, test_y)

print('LINEAR CLASSIFIER')
classifier = LinearClassifier(training_x, training_y, test_x, test_y, manager.get_classes_length())

RAW DATA
[['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'], ['5.1', '3.5', '1.4', '0.2', 'Iris-setosa'], ['4.9', '3', '1.4', '0.2', 'Iris-setosa'], ['4.7', '3.2', '1.3', '0.2', 'Iris-setosa'], ['4.6', '3.1', '1.5', '0.2', 'Iris-setosa'], ['5', '3.6', '1.4', '0.2', 'Iris-setosa'], ['5.4', '3.9', '1.7', '0.4', 'Iris-setosa'], ['4.6', '3.4', '1.4', '0.3', 'Iris-setosa'], ['5', '3.4', '1.5', '0.2', 'Iris-setosa'], ['4.4', '2.9', '1.4', '0.2', 'Iris-setosa'], ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'], ['5.4', '3.7', '1.5', '0.2', 'Iris-setosa'], ['4.8', '3.4', '1.6', '0.2', 'Iris-setosa'], ['4.8', '3', '1.4', '0.1', 'Iris-setosa'], ['4.3', '3', '1.1', '0.1', 'Iris-setosa'], ['5.8', '4', '1.2', '0.2', 'Iris-setosa'], ['5.7', '4.4', '1.5', '0.4', 'Iris-setosa'], ['5.4', '3.9', '1.3', '0.4', 'Iris-setosa'], ['5.1', '3.5', '1.4', '0.3', 'Iris-setosa'], ['5.7', '3.8', '1.7', '0.3', 'Iris-setosa'], ['5.1', '3.8', '1.5', '0.3', 'Iris-setosa'], ['5.4', '3.4', '1.7', '0.2'

##TEST #1 - SPRAWDZENIE MNOŻENIA MACIERZOWEGO 

In [ ]:
print('TEST #1')
print('BY DEFAULT WEIGHT MATRIX IS NUMPY ONES, SO EVERY CLASS SHOULD HAVE THE SAME SCORE')
print('INSTANCE:', training_x[0])
print('SCORE:', classifier.get_score(training_x[0]))

TEST #1
BY DEFAULT WEIGHT MATRIX IS NUMPY ONES, SO EVERY CLASS SHOULD HAVE THE SAME SCORE
INSTANCE: [5.1 3.8 1.9 0.4 1. ]
SCORE: [12.2 12.2 12.2]


##TEST #2 - CROSS-ENTROPY LOSS FUNCTION

In [ ]:
print('TEST #2')
print('CHECK MATH OPERATIONS OF THE CROSS-ENTROPY LOSS FUNCTION')
cross_entropy_loss_function(training_x[0], classifier.W, training_y[0], debug=True)

TEST #2
CHECK MATH OPERATIONS OF THE CROSS-ENTROPY LOSS FUNCTION
SCORE: [12.2 12.2 12.2]
e^s:  [198789.15114295 198789.15114295 198789.15114295]
sum of the e^s: 596367.4534288628
normalized probability: 0.3333333333333333
loss: 1.0986122886681098


1.0986122886681098

##TEST #3 - LOSS FUNCTION DLA ZESTAWU DANYCH

In [ ]:
print('TEST #3')
print('CHECK LOSS FUNCTION FOR THE DATASET')
print('LOSS:', loss_function_for_the_dataset(training_x, classifier.W, training_y))

TEST #3
CHECK LOSS FUNCTION FOR THE DATASET
LOSS: 1.098612288668108


##TEST #4 - REGULARYZACJA L2

In [ ]:
print('TEST #4')
print('REGULARIZATION L2')
print('CHECK LOSS WILL BE SMALLER FOR MORE \'SPREAD OUT\' WEIGHT MATRIX')
w1 = np.zeros(classifier.W.shape)
w1[:, 0] = 1.0
print('W:', w1)
print('LOSS:', loss_function_for_the_dataset(training_x, w1, training_y))
print('LOSS L2:', loss_function_for_the_dataset_with_L2(training_x, w1, training_y))
w2 = np.full(classifier.W.shape, 0.25)
print('W:', w2)
print('LOSS:', loss_function_for_the_dataset(training_x, w2, training_y))
print('LOSS L2:', loss_function_for_the_dataset_with_L2(training_x, w2, training_y))

TEST #4
REGULARIZATION L2
CHECK LOSS WILL BE SMALLER FOR MORE 'SPREAD OUT' WEIGHT MATRIX
W: [[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
LOSS: 1.098612288668108
LOSS L2: 1.128612288668108
W: [[0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25]]
LOSS: 1.098612288668108
LOSS L2: 1.107987288668108


##TEST #5 - LOSOWE SZUKANIE MACIERZY WAG

In [ ]:
print("TEST #5")
print('NAIVE RANDOM WEIGHT MATRIX SEARCH')
print('ALWAYS STARTS WITH NUMPY ONES WEIGHT MATRIX')
for i in range(200, 1200, 200):
    print(i, 'ITERATIONS')
    w, loss = classifier.random_search(iterations=i)
    accuracy, options = classifier.check_accuracy()
    print('loss:', loss, 'accuracy:', accuracy)
    print('chosen classes:', options)
log_c = np.log(manager.get_classes_length())
print('LOG(C):', log_c)

TEST #5
NAIVE RANDOM WEIGHT MATRIX SEARCH
ALWAYS STARTS WITH NUMPY ONES WEIGHT MATRIX
200 ITERATIONS
loss: 1.0949814828881896 accuracy: 0.6333333333333333
chosen classes: [9, 0, 21]
400 ITERATIONS
loss: 1.0954383956715634 accuracy: 0.3333333333333333
chosen classes: [0, 0, 30]
600 ITERATIONS
loss: 1.0949100021082028 accuracy: 0.3333333333333333
chosen classes: [0, 0, 30]
800 ITERATIONS
loss: 1.095404272513766 accuracy: 0.3333333333333333
chosen classes: [0, 0, 30]
1000 ITERATIONS
loss: 1.0953411168358358 accuracy: 0.3333333333333333
chosen classes: [0, 0, 30]
LOG(C): 1.0986122886681098


##TEST #6 - GRADIENT DESCENT DLA 1000 ITERACJI (GRADIENT LICZONY NUMERYCZNIE)

In [ ]:
print('TEST #6')
print('GRADIENT DESCENT WITH NUMERIC GRADIENT')
print('1000 ITERATIONS')
w, loss = classifier.gradient_descent()
accuracy, options = classifier.check_accuracy()
print('loss:', loss, 'accuracy:', accuracy)
print('chosen classes:', options)

TEST #6
GRADIENT DESCENT WITH NUMERIC GRADIENT
1000 ITERATIONS
loss: 0.5248221911393842 accuracy: 1.0
chosen classes: [10, 10, 10]


##TEST #7 - ADAM DLA 1000 ITERACJI (GRADIENT LICZONY NUMERYCZNIE)

In [ ]:
print('TEST #7')
print('ADAM WITH NUMERIC GRADIENT')
print('1000 ITERATIONS')
w, loss = classifier.adam()
accuracy, options = classifier.check_accuracy()
print('loss:', loss, 'accuracy:', accuracy)
print('chosen classes:', options)

TEST #7
ADAM WITH NUMERIC GRADIENT
1000 ITERATIONS
loss: 0.5500518833670252 accuracy: 1.0
chosen classes: [10, 10, 10]
